In [13]:
# Install required packages.
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install sentence-transformers
import torch
import pandas as pd
from torch_geometric.data import HeteroData, download_url, extract_zip
from torch_geometric.transforms import ToUndirected, RandomLinkSplit
# Helper functions for visualization.
%matplotlib inline

You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\1\PycharmProjects\pythonProject4\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [14]:

clinical = pd.read_csv('data/clinical_data_breast_cancer.csv', index_col=0)
proteomes_orig = pd.read_csv('data/77_cancer_proteomes_CPTAC_itraq.csv')
PAM50 = pd.read_csv('data/PAM50_proteins.csv')
proteomes = proteomes_orig.drop(['gene_symbol','gene_name'], axis=1)
clinical.index = clinical.index.to_series().apply(lambda title : title.split('CGA-')[1])
proteomes.rename(columns = proteomes.columns.to_series().apply(lambda title: title.split('.')[0]), inplace=True)

#Transpose and organize proteomes data
proteomes = proteomes.transpose()
proteomes.columns =  proteomes.iloc[0]
proteomes.drop('RefSeq_accession_number', axis=0, inplace=True)

#Convert gender to numbers
def num_gender(gender):
    if gender == 'MALE':
        return 0
    elif gender == 'FEMALE':
        return 1
    else:
        return float('NaN')

clinical['Gender'] = clinical['Gender'].apply(lambda gender: num_gender(gender))

#Convert status to numbers
def num_status(status):
    if status == 'Negative':
        return 0
    elif status == 'Positive':
        return 1
    else:
        return

clinical['ER Status'] = clinical['ER Status'].apply(lambda status: num_status(status))
clinical['PR Status'] = clinical['PR Status'].apply(lambda status: num_status(status))
clinical['HER2 Final Status'] = clinical['HER2 Final Status'].apply(lambda status: num_status(status))

#Convert tumor, node, metastasis to numbers
clinical['Tumor'] = clinical['Tumor'].apply(lambda tumor: tumor.split('T')[1])
clinical['Node'] = clinical['Node'].apply(lambda tumor: tumor.split('N')[1])
clinical['Metastasis'] = clinical['Metastasis'].apply(lambda tumor: tumor.split('M')[1])
#Remove unused columns
clinical.drop('Tumor--T1 Coded', axis=1, inplace=True)
clinical.drop('Metastasis-Coded', axis=1, inplace=True)
clinical.drop('Node-Coded', axis=1, inplace=True)

#Merge clinical data with proteomes data
datasetPrim = clinical.merge(proteomes, left_index=True,right_index=True)

datasetPrim

,Gender,Age at Initial Pathologic Diagnosis,ER Status,PR Status,HER2 Final Status,Tumor,Node,Metastasis,AJCC Stage,Converted Stage,...,NP_001193600,NP_061134,NP_932347,NP_003593,NP_997203,NP_001191293,NP_775791,NP_004065,NP_068752,NP_219494
A2-A0CM,1,40,0.0,0,0.0,2,0,0,Stage IIA,Stage IIA,...,NaN,NaN,1.153614,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A2-A0D2,1,45,0.0,0,0.0,2,0,0,Stage IIB,Stage IIA,...,0.919136,-1.648856,0.832649,NaN,-8.324969,-4.679219,NaN,-1.10665,NaN,-6.941181
A2-A0EQ,1,64,0.0,0,1.0,2,0,0,Stage IIA,Stage IIA,...,-0.801685,NaN,NaN,3.80231,-6.373934,-1.12316,NaN,NaN,NaN,NaN
A2-A0EV,1,80,1.0,1,0.0,1,0,0,Stage IA,Stage I,...,-4.966177,-1.471027,NaN,-0.474013,-12.278546,-10.337729,-0.653251,NaN,NaN,NaN
A2-A0EX,1,46,1.0,1,0.0,3,0,0,Stage IIB,Stage IIB,...,1.45149,-2.018981,0.877456,NaN,NaN,-6.101005,NaN,-1.726336,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C8-A138,1,54,1.0,0,1.0,2,2,0,Stage III,Stage IIIA,...,-3.250913,1.711825,-0.248402,NaN,4.707022,4.107251,-3.20337,1.971481,NaN,NaN
D8-A142,1,74,0.0,0,0.0,3,0,0,Stage IIB,Stage IIB,...,-5.107629,-0.97598,NaN,2.508629,-12.33711,-9.54653,-4.066584,NaN,NaN,NaN
E2-A154,1,68,1.0,1,0.0,1,0,0,Stage I,Stage I,...,-3.386203,-2.328692,-2.806642,NaN,-4.733495,-9.584499,-4.786183,-3.103949,NaN,NaN
E2-A158,1,43,0.0,0,0.0,1,1,0,Stage IIA,Stage IIA,...,-0.638364,0.051811,2.509998,7.067839,NaN,0.378972,NaN,NaN,0.665797,NaN


In [15]:
import numpy as np

noNaProteasomesColumns= np.array(datasetPrim.dropna(axis=1).columns[24:])
columnsOfIntrest = np.insert(noNaProteasomesColumns, 0, 'AJCC Stage', axis=0)
datasetPrim[columnsOfIntrest]

,AJCC Stage,NP_958782,NP_958785,NP_958786,NP_000436,NP_958781,NP_958780,NP_958783,NP_958784,NP_112598,...,NP_001652,NP_071753,NP_001294,NP_065775,NP_001094348,XP_003846524,NP_443073,NP_001004456,NP_997625,NP_003270
A2-A0CM,Stage IIA,0.683404,0.694424,0.698098,0.687077,0.687077,0.698098,0.698098,0.698098,-2.65215,...,0.61728,-1.502339,2.130769,1.098511,-0.819065,-0.675798,0.014823,1.550354,-0.216608,1.032388
A2-A0D2,Stage IIB,0.107491,0.104164,0.107491,0.097512,0.104164,0.104164,0.104164,0.104164,-0.880454,...,2.136604,2.322883,1.225166,1.777351,1.062172,-2.759878,0.154061,0.237221,0.270485,0.456764
A2-A0EQ,Stage IIA,-0.91267,-0.927979,-0.927979,-0.931806,-0.927979,-0.927979,-0.927979,-0.927979,-3.071151,...,2.133697,1.643829,1.364451,2.481962,1.785431,-1.203529,0.664093,0.289037,-0.047747,0.128299
A2-A0EV,Stage IA,0.452986,0.47259,0.47259,0.458587,0.47259,0.47259,0.47259,0.47259,-0.742871,...,1.335175,1.884093,2.231367,2.463817,-3.078572,-6.167635,0.842269,4.158181,-0.160346,-2.32521
A2-A0EX,Stage IIB,1.185108,1.192612,1.18886,1.185108,1.200116,1.18886,1.18886,1.192612,1.046289,...,2.130577,2.3782,2.509515,0.948741,0.108324,-0.743348,0.787411,3.158587,2.948483,2.025525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C8-A138,Stage III,2.765081,2.779709,2.779709,2.797995,2.787023,2.779709,2.783366,2.783366,2.205538,...,-3.426456,-1.733201,-0.584859,-1.18463,2.110453,1.496053,-1.718572,0.197769,-2.354914,-0.354459
D8-A142,Stage IIB,0.538596,0.542211,0.542211,0.534981,0.542211,0.542211,0.542211,0.542211,-0.148205,...,1.756763,4.352147,2.259212,1.247085,0.835004,-1.149488,2.204991,4.070197,2.251983,-0.556671
E2-A154,Stage I,0.862659,0.870186,0.870186,0.866423,0.870186,0.870186,0.870186,0.870186,1.920171,...,1.64168,-0.409365,-1.73784,-1.116882,-1.135699,-3.694801,0.873949,4.618518,0.636856,-0.488396
E2-A158,Stage IIA,-1.086529,-1.095492,-1.095492,-1.095492,-1.095492,-1.093252,-1.093252,-1.093252,0.096627,...,1.358213,0.338637,2.301601,-0.371705,1.909456,0.018198,-0.0311,0.282616,-0.400836,2.492071


In [16]:
#loads dataframe and apply encoder
def load_node_csv(df, encoders=None):
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

#load edges based on csv ...
def load_edge_csv( df,src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None):
    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])
    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)
    return edge_index, edge_attr

In [17]:
#can encode the sentence in this case we can use it for example for histopathological names ...
from sentence_transformers import SentenceTransformer

class SequenceEncoder(object):
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

In [18]:
#getting mappings of variables of intrest
AjCC_x,AjCC_mapping = load_node_csv(datasetPrim[['AJCC Stage']],encoders={
        'AJCC Stage': SequenceEncoder()
    } )
proteasomes_x,proteasomes_mapping = load_node_csv(datasetPrim[noNaProteasomesColumns])


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
datasetPrim[['AJCC Stage']]

,AJCC Stage
A2-A0CM,Stage IIA
A2-A0D2,Stage IIB
A2-A0EQ,Stage IIA
A2-A0EV,Stage IA
A2-A0EX,Stage IIB
...,...
C8-A138,Stage III
D8-A142,Stage IIB
E2-A154,Stage I
E2-A158,Stage IIA


In [20]:
torch.tensor(np.unique(np.array(datasetPrim['AJCC Stage'])))

array(['Stage I', 'Stage IA', 'Stage IB', 'Stage II', 'Stage IIA',
       'Stage IIB', 'Stage III', 'Stage IIIA', 'Stage IIIB', 'Stage IIIC',
       'Stage IV'], dtype=object)

In [47]:
np.unique(datasetPrim['AJCC Stage'].values)

array(['Stage I', 'Stage IA', 'Stage IB', 'Stage II', 'Stage IIA',
       'Stage IIB', 'Stage III', 'Stage IIIA', 'Stage IIIB', 'Stage IIIC',
       'Stage IV'], dtype=object)

In [48]:
torch.arange(11)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [29]:
from torch_geometric.data import HeteroData
# hgraph = {
#     "proteasomes": {
#         "num_nodes": len(datasetPrim)
#     },
#     "stage":{
#         "num_nodes": len(np.unique(np.array(datasetPrim['AJCC Stage'])))
#     }
# }
data = HeteroData()
data['proteasomes'].x=torch.arange(11) #11 becouse we have 11 diffrent stages
# data['proteasomes'].x = [len(datasetPrim),len(noNaProteasomesColumns)]
# data['proteasomes'].x
data['proteasomes']
# node_types, edge_types = data.metadata()
# node_types, edge_types

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [27]:
#helper example from https://github.com/www602/RD_GAT/blob/05b904c1d3477a91a6d2f7c92515c3fc3e4f4d52/3.metapath2vec/1.AMiner2Vec.py
from torch_geometric.data import HeteroData

data = HeteroData(hgraph)
data['aa'].z = torch.tensor([1,2,3])
data['aa'].x = torch.tensor([1,2,3])
data.z_dict
data.x_dict
# data.metadata()

AttributeError: 'BaseStorage' object has no attribute 'todict'

In [23]:
import torch
import networkx as nx
import matplotlib.pyplot as plt

def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()
from torch_geometric.utils import to_networkx

G = to_networkx(data)
visualize_graph(G)

TypeError: 'HeteroData' object is not callable

In [25]:
authors = torch.rand((10,8))
papers = torch.rand((20,4))
authors_y = torch.rand(10).round()

In [ ]:







class SequenceEncoder(object):
    # The 'SequenceEncoder' encodes raw column strings into embeddings.
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()


class GenresEncoder(object):
    # The 'GenreEncoder' splits the raw column strings by 'sep' and converts
    # individual elements to categorical labels.
    def __init__(self, sep='|'):
        self.sep = sep

    def __call__(self, df):
        genres = set(g for col in df.values for g in col.split(self.sep))
        mapping = {genre: i for i, genre in enumerate(genres)}

        x = torch.zeros(len(df), len(mapping))
        for i, col in enumerate(df.values):
            for genre in col.split(self.sep):
                x[i, mapping[genre]] = 1
        return x


class IdentityEncoder(object):
    # The 'IdentityEncoder' takes the raw column values and converts them to
    # PyTorch tensors.
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)


user_x, user_mapping = load_node_csv(rating_path, index_col='userId')

movie_x, movie_mapping = load_node_csv(
    movie_path, index_col='movieId', encoders={
        'title': SequenceEncoder(),
        'genres': GenresEncoder()
    })

edge_index, edge_label = load_edge_csv(
    rating_path,
    src_index_col='userId',
    src_mapping=user_mapping,
    dst_index_col='movieId',
    dst_mapping=movie_mapping,
    encoders={'rating': IdentityEncoder(dtype=torch.long)},
)

data = HeteroData()
data['user'].num_nodes = len(user_mapping)  # Users do not have any features.
data['movie'].x = movie_x
data['user', 'rates', 'movie'].edge_index = edge_index
data['user', 'rates', 'movie'].edge_label = edge_label
print(data)

# We can now convert `data` into an appropriate format for training a
# graph-based machine learning model:

# 1. Add a reverse ('movie', 'rev_rates', 'user') relation for message passing.
data = ToUndirected()(data)
del data['movie', 'rev_rates', 'user'].edge_label  # Remove "reverse" label.

# 2. Perform a link-level split into training, validation, and test edges.
transform = RandomLinkSplit(
    num_val=0.05,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('user', 'rates', 'movie')],
    rev_edge_types=[('movie', 'rev_rates', 'user')],
)
train_data, val_data, test_data = transform(data)
print(train_data)
print(val_data)
print(test_data)